<a id='top'></a>

# Object Detection with Tensorflow

This notebook uses a pre-trained model (ssd_mobilenet_v1_coco_11_06_2017) and detect objects upto 90 classes.
Have a look how to use tensorflow to build object detection app for detecting multiple objects on any jpg and png image.

### Here is the code

<hr>

## importing important libraries

In [ ]:
# Import packages
import os
import cv2
import numpy as np
import tensorflow as tf
import sys
import time

## other imports from module

In [ ]:
# Import utilites
from utils import label_map_util
from utils import visualization_utils as vis_util

# Model preparation 

## Variables

In [ ]:
# Path to frozen detection graph.
PATH_TO_CKPT = 'frozen_inference_graph.pb'

# strings that is used to add correct label for each box.
PATH_TO_LABELS = 'mscoco_label_map.pbtxt'

NUM_CLASSES = 90

In [ ]:
# Path to image
PATH_TO_TEST_IMAGES_DIR = 'test_images'
TEST_IMAGE_PATHS = [ os.path.join(CWD_PATH,PATH_TO_TEST_IMAGES_DIR, 'lite{}.jpg'.format(i)) for i in range(39,40) ]

## Loading label map
Label maps map indices to category names, so that when our convolution network predicts `1`, we know that this corresponds to `car`.

In [ ]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Loading  a Tensorflow model into memory.

In [ ]:
# Load the Tensorflow model into memory.
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

    sess = tf.Session(graph=detection_graph)

# Detection

In [ ]:
# Input tensor is the image
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')

# Output tensors are the detection boxes, scores, and classes
# Each box represents a part of the image where a particular object was detected
detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')

# Each score represents level of confidence for each of the objects.
# The score is shown on the result image, together with the class label.
detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')

# Number of objects detected
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

In [ ]:
for image_path in TEST_IMAGE_PATHS:
    # Load image using OpenCV and
    # expand image dimensions to have shape: [1, None, None, 3]
    # i.e. a single-column array, where each item in the column has the pixel RGB value
    image = cv2.imread(image_path)

    
    t = time.time()
    
    
    
    cv2.namedWindow("output", cv2.WINDOW_NORMAL) 
    
    image_expanded = np.expand_dims(image, axis=0)

    # Perform the actual detection by running the model with the image as input
    (boxes, scores, classes, num) = sess.run(
        [detection_boxes, detection_scores, detection_classes, num_detections],
        feed_dict={image_tensor: image_expanded})

    # Draw the results of the detection (aka 'visulaize the results')

    vis_util.visualize_boxes_and_labels_on_image_array(
        image,
        np.squeeze(boxes),
        np.squeeze(classes).astype(np.int32),
        np.squeeze(scores),
        category_index,
        use_normalized_coordinates=True,
        line_thickness=2,
        max_boxes_to_draw=400,
        min_score_thresh=0.80)
    
    # All the results have been drawn on image. Now save the image.
    cv2.imwrite("result{}.jpg".format(val), image)  
    
    print('[INFO] elapsed time: {:.2f}'.format(time.time() - t))
    
    
    
    height, width, _ = image.shape
    i=0
    s=image_path.split('\\')[-1]
    val=s[:len(s)-4]
    file=open('predicted/{}.txt'.format(val),'w')
    for index, value in enumerate(classes[0]):
        if (scores[0,index] > 0.5):
            ymin = boxes[0][i][0]*height
            xmin = boxes[0][i][1]*width
            ymax = boxes[0][i][2]*height
            xmax = boxes[0][i][3]*width
            file.write('{} {} {} {} {} {} \n'.format(category_index.get(value)['name'],scores[0, index],int(xmin), int(ymin), int(xmax), int(ymax)))
            i += 1
    file.close()

<hr>

<b><a href='#top'>Return to top</a></b>

This application of object detection is also given in the official github page of tensorflow for object detection.

<hr>